In [1]:
!pip install ultralytics

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/Desktop/SatellieteIMGDetection
import os
print(os.getcwd())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection


In [ ]:
import json

# 하이퍼파라미터 튜닝
from ultralytics import YOLO
model = YOLO("yolo11x-obb.pt")
best_params = model.tune(
    data="data_combine.yaml",
    batch=-1,
    epochs=15,
    iterations=50,
    optimizer="AdamW",
    plots=True,
    save=False,   # 모든 iteration에서 저장하지 않도록 설정
    val=True,
    lr0=0.001,
    scale=0.5,
)

# 최적 하이퍼파라미터 저장
with open("best_params.json", "w") as f:
    json.dump(best_params, f)
print("Best parameters saved to best_params.json.")


Tuner: Initialized Tuner instance with 'tune_dir=runs\obb\tune'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/50 with hyperparameters: {'lr0': 0.001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


In [1]:

# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt")
results = model.train(
    data="data_combine.yaml",
    epochs=20,
    batch=3,
    imgsz=1024,
    patience=15,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00088,
    lrf= 0.01134,
    momentum= 0.82858,
    weight_decay= 0.0006,
    warmup_epochs= 2.33952,
    warmup_momentum= 0.58988,
    box= 5.06927,
    cls= 0.56241,
    dfl= 1.36767,
    hsv_h= 0.01707,
    hsv_s= 0.68688,
    hsv_v= 0.28102,
    degrees= 0.0,
    translate= 0.11263,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.45936,
    fliplr= 0.46892,
    bgr= 0.1,
    mosaic= 0.828,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.5,
    dropout = 0.2,
    )


Ultralytics 8.3.12  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_combine.yaml, epochs=20, time=None, patience=15, batch=3, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 864 images, 140 backgrounds, 0 corrupt: 100%|██████████| 864/864 [00:00<?, ?it/s]
val: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]


Plotting labels to runs\obb\train9\labels.jpg... 
optimizer: AdamW(lr=0.00088, momentum=0.82858) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.000590625), 182 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs\obb\train9
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      10.8G     0.9298      2.215      1.306        145       1024: 100%|██████████| 288/288 [02:02<00:00,  2.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  5.52it/s]

                   all         92       4910      0.671      0.613      0.625       0.43


...

      20/20      12.4G     0.7415      2.268      1.163        546       1024: 100%|██████████| 288/288 [01:57<00:00,  2.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  5.69it/s]

                   all         92       4910      0.807       0.69      0.756      0.568



20 epochs completed in 0.702 hours.
Optimizer stripped from runs\obb\train9\weights\last.pt, 118.4MB
Optimizer stripped from runs\obb\train9\weights\best.pt, 118.4MB

Validating runs\obb\train9\weights\best.pt...
Ultralytics 8.3.12  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:06<00:00,  4.77it/s]


                   all         92       4910      0.806       0.69      0.756      0.568
               ship(S)         79       4334      0.869       0.68      0.756      0.548
               ship(L)         61        576      0.743        0.7      0.755      0.588
Speed: 0.9ms preprocess, 39.0ms inference, 0.0ms loss, 8.7ms postprocess per image
Results saved to runs\obb\train9


In [3]:
import torch

# CUDA 버전 확인
cuda_version = torch.version.cuda
print(f"CUDA Version: {cuda_version}")

# CUDA 연결 확인
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    # GPU 이름 확인
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {gpu_name}")
else:
    print("CUDA is not available. Please check your CUDA installation.")

CUDA Version: 12.1
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Ti
